Attrition Prediction Exercise - Data Wrangling

This project is aimed at understanding what factors in an employee's current work environment could be influencing their decision to leave the company.

In order to do this analysis, i will be bringing together:

-- employee data available in the Human Capital Management (HCM) system - this includes the employee's job level, salary, compa-ratio (salary in comparison to mid point for that role in that location), gender etc.

-- employee's performance data

-- employee's reasons for leaving

-- employee's response to the last engagement survey before they left the organization

Please note: I have created and used a PII safe dataset. Because this information is based on my current organization, i am not uploading the data (even though it is PII safe) in the git repo.

In [1]:
#start by importing any libraries i might need
!pip install matplotlib
!pip install seaborn

In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
#Next import all the csv files containing all the employee data we will need and will be merging 
#emp_df includes all data about employee from the Human Capital Management system including job level, salary, gender etc.
emp_df = pd.read_csv(r'C:\Users\m.bajaj\Python\Capstone2_Attrition\PII_safe_data\emp_data_no_pii.csv')
#perf_df includes data about employee's last recieved performance rating
perf_df = pd.read_csv(r'C:\Users\m.bajaj\Python\Capstone2_Attrition\PII_safe_data\perf_data_no_pii_use.csv')
#term_df includes additional data about termination like reason for leaving 
term_df = pd.read_csv(r'C:\Users\m.bajaj\Python\Capstone2_Attrition\PII_safe_data\term_data_no_pii.csv')
#engmt_df includes data from the last engagement survey taken by the employee. This spans over about 47 questions rated on a 5 point likert scale
engmt_df = pd.read_csv(r'C:\Users\m.bajaj\Python\Capstone2_Attrition\PII_safe_data\engmt_data_no_pii_use_final.csv')

In [4]:
# Start by looking at the emp_df dataset 
#emp_df.head()

In [5]:
emp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543 entries, 0 to 542
Data columns (total 32 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Title                                            543 non-null    object 
 1   Worker Type (regular, international, temporary)  543 non-null    object 
 2   Employment Type                                  543 non-null    object 
 3   Department                                       543 non-null    object 
 4   Work Location                                    543 non-null    object 
 5   Compensation (hourly wage or salary)             543 non-null    float64
 6   Compensation type (hourly, salary)               543 non-null    object 
 7   Currency                                         543 non-null    object 
 8   Current Employment Start Date                    543 non-null    object 
 9   Initial Employment Start Date   

Notice in the above data that employee id is an integer, manager id is a float and date fields are objects. Need to rectify them. Will do this once i merge all data sets.

In [6]:
#count missing values by column
missing = pd.concat([emp_df.isna().sum(), 100 * emp_df.isna().mean()], axis = 1)
missing.columns = ['count', '%']
missing.sort_values(by = '%', ascending = False)

,count,%
Unnamed: 31,543,100.000000
Termination Date,456,83.977901
EEO Job Category,266,48.987109
EEO Race,263,48.434622
State,18,3.314917
Comp Mid Point,15,2.762431
City,8,1.473297
Job Code,5,0.920810
Date of Birth,2,0.368324
Age,2,0.368324


EEEO Race and Job Category is missing a lot of values. This is because it is an optional data field for employees to fill. We will keep this as is for now

Termination date is missing for a significant amount of people. This is actually correct because most employees in our file are active employees and only 87 are terminated which is why we have termination dates only for 87 employees

City and State information is missing for a few people - this is because we have a few employees that are outside US and do not have this level of dat available. We will keep this as is for now

Comp mid-point is missing for 15 people. All 15 people are part of the executive team and mid-point is not available for them.

Lastly, in a prior iteration, it was realized that data was missing for FSLA status, Job ID etc. This was fixed in the raw data file and hence is not visible in version of data wrangling.

In [7]:
#Next we will review the data in our performance data frame
#perf_df.head()

In [8]:
perf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543 entries, 0 to 542
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Employee ID              543 non-null    int64 
 1   Performance Rating Date  385 non-null    object
 2   Performance Rating       384 non-null    object
dtypes: int64(1), object(2)
memory usage: 12.9+ KB


We see that there are no missing values for performance rating and employee id columns so we are good to go ahead and merge this data with our emp_df dataframe.

In [9]:
#New we will merge emp_df with perf_df
emp_df = pd.merge(emp_df, perf_df, how = 'left', on = 'Employee ID')
#emp_df.head()

We reviewed the combined dataframe and it looked good. So we move to the next step i.e. reviewing and merging our next data frame called term_df

In [10]:
#Now we review our term dataframe
#term_df.head()

In [11]:
#Reviewing to see if we are missing any values on our term_df dataframe
missing_term = pd.concat([term_df.isna().sum(), 100* term_df.isna().mean()], axis = 1)
missing_term.columns = ['count', '%']
missing_term.sort_values(by = '%', ascending = False)

,count,%
Source,72,80.898876
Title or Role at New Company,49,55.056180
Company Employee is going?,35,39.325843
Termination Notes,22,24.719101
Term Reason,5,5.617978
Tenure Months,2,2.247191
Tenure,0,0.000000
Term Type,0,0.000000
Employee ID,0,0.000000


After the review, we saw term reason was missing for a few employees. We went back and fixed this in the raw data. Term reason for 5 employees is actually not available and we will keep it as is.

We see that 'Title or Role at New Company' and 'Company Employee is going?' has a number of missing values. These fields do not appear to be critical to our analysis. We will merge the data frames and then drop these columns

In [12]:
#New merge term with emp dataframe
emp_df = pd.merge(emp_df, term_df, how = 'left', on = 'Employee ID')

In [13]:
emp_df = emp_df.drop(['Title or Role at New Company', 'Company Employee is going?', 'Tenure Months', 'Tenure'], axis = 1)

In [14]:
#emp_df.head()

In [15]:
#Next review engagement data
#engmt_df.head()

Initially, i had imported survey data only for the month of December 2022. However, on review it was realized that survey information was missing for a large number of people (300+). To overcome the issue, i then acquired data for the last survey taken by employees. This reduced the missing information greatly. This was however updated at the raw data stage.

In [16]:
# Next merge emp_df with engmt_df
emp_df = pd.merge(emp_df, engmt_df, how = 'left', on = 'Employee ID')
#emp_df.head()

In [17]:
#We now have a very big data set and most columns are hidden. Writing a code to unhide all hidden columns in between. This is particularly needed as we explore .info()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [18]:
#emp_df.head()

In [19]:
#count missing values by column
missing = pd.concat([emp_df.isna().sum(), 100 * emp_df.isna().mean()], axis = 1)
missing.columns = ['count', '%']
missing.sort_values(by = '%', ascending = False)

,count,%
Unnamed: 31,543,100.000000
Source,527,97.053407
Termination Notes,481,88.581952
Term Reason,464,85.451197
Term Type,459,84.530387
Termination Date,456,83.977901
EEO Job Category,266,48.987109
EEO Race,263,48.434622
Performance Rating,159,29.281768
Performance Rating Date,158,29.097606


We see the fields identified earlier i.e. EEO job category etc are still the same in terms of missing values. As decided, we will not do anything with these fieldss at this stage. Also, there is little scope to acquire this information.

Performance rating and survey data is missing for about 150+ employees. However, this is a lot less than wheat we had initially. As mentioned, the larger number of missing values were fixed by taking the last performance rating and last survey taken for all employees. No more action can be taken at this point and this information is relevant and we would like to retain it at this point to better understand attrition.

In [20]:
emp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543 entries, 0 to 542
Data columns (total 89 columns):
 #   Column                                                                                                                      Non-Null Count  Dtype  
---  ------                                                                                                                      --------------  -----  
 0   Title                                                                                                                       543 non-null    object 
 1   Worker Type (regular, international, temporary)                                                                             543 non-null    object 
 2   Employment Type                                                                                                             543 non-null    object 
 3   Department                                                                                                                  5

We notice most date fields are 'objects', 'comp Mid Point' is an object and 'Manager Id' is a float. We will work on fixing these data types next.

In [21]:
#change all date fields to datetime dtype and manager id to an integer
emp_df [['Current Employment Start Date', 'Initial Employment Start Date', 'Termination Date', 'Date of Birth', 'Performance Rating Date', 'survey_date']] = emp_df [['Current Employment Start Date', 'Initial Employment Start Date', 'Termination Date', 'Date of Birth', 'Performance Rating Date', 'survey_date']].apply(pd.to_datetime)
emp_df['Manager ID'] = emp_df['Manager ID'].astype(int, errors='ignore')
emp_df['Comp Mid Point'] = pd.to_numeric(emp_df['Comp Mid Point'], errors = 'coerce')
emp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543 entries, 0 to 542
Data columns (total 89 columns):
 #   Column                                                                                                                      Non-Null Count  Dtype         
---  ------                                                                                                                      --------------  -----         
 0   Title                                                                                                                       543 non-null    object        
 1   Worker Type (regular, international, temporary)                                                                             543 non-null    object        
 2   Employment Type                                                                                                             543 non-null    object        
 3   Department                                                                                

As a next step, we also need to do some calculations which will be helpful in our analysis

First we, need to make sure that we have annual salary for all employees. Current salary column shows annual salary for those with annual salary but only an hourly amount for the hourly paid employees.

Next, since we want to do a survival curve analysis to predict attrition, we need to calcular tenure. We will calculate tenure in weeks, given most people have been at the organization of interest for less than a year.

Finally, we want to see how are people being paid in comparison to market. We will compare people's salary to market mid-point and create a field called 'compa-ratio-%'

In [22]:
#First, we will calculate annual compensation. We need to use two different attributes to calculate this field. 
#One attribute telling us whether someone's salary is hourly, and 
#Second attribute being compensation amount

#emp_df['annual_salary'] = emp_df['Compensation (hourly wage or salary)']
#emp_df['annual_salary'][emp_df['Compensation type (hourly, salary)'] == 'H'] = emp_df['Compensation (hourly wage or salary)'][emp_df['Compensation type (hourly, salary)'] == 'H']*2080
#emp_df

emp_df['annual_salary'] = np.where(emp_df['Compensation type (hourly, salary)'] == 'H',emp_df['Compensation (hourly wage or salary)'] * 2080, 
                                   emp_df['Compensation (hourly wage or salary)'])
                                   

Next, we want to calculate tenure. For this we first need to add today's date as a new column and make sure the new column is datetime data type

In [23]:
#Add a column with today's date to calculate tenure
emp_df['todays date']= datetime.today().strftime('%Y-%m-%d')
emp_df['todays date'] = pd.to_datetime(emp_df['todays date'])

In [24]:
print(emp_df.dtypes)

Title                                                                                                                                 object
Worker Type (regular, international, temporary)                                                                                       object
Employment Type                                                                                                                       object
Department                                                                                                                            object
Work Location                                                                                                                         object
Compensation (hourly wage or salary)                                                                                                 float64
Compensation type (hourly, salary)                                                                                                    object
Currency     

Now we will calculate tenure. However, in our dataframe we have both active and terminated employees. For all active employees we can calculate tenure by subtracting today's date from their hire date.

However, for terminated employees, we need to calculate tenure by subtracting their termination date from their hire date.

I used the np.where() function to derive this value

In [25]:
#Use the np.where() function to calculate tenure in weeks
emp_df['Tenure_weeks'] = np.where(emp_df['Status (Active / Terminated)'] == 'Active', 
                                  (emp_df['todays date'] - emp_df['Current Employment Start Date'])/np.timedelta64(1,'W'), 
                                  (emp_df['Termination Date'] - emp_df['Current Employment Start Date']) /np.timedelta64(1, 'W'))
#emp_df.head()

Next, we calculate the compa-ratio in percentage which is how does one's annual salary compare with the market mid point for that role in that given location.

In [26]:
#Compa ratio is merely a ratio between the annual salary and market mid-point and can be calculated by:
#dividing annual salary by comp mid point.  
#Additionally, we will also multiply this number by 100 so we get a percentage
emp_df['compa-ratio-%'] = (emp_df['annual_salary']/emp_df['Comp Mid Point'])*100
#emp_df.head()

In [27]:
#reviewing the data ranges for numerical data types
#emp_df.describe()

We do not see any anomolies in the distribution of the data at the moment. 

Employee and Manager ID show up as a float although they are not. However, we will not change this at this point.

Lastly, we will export this data as a comma separated values (csv) file and and this new file will become our final dataset for futher steps in the analysis

In [28]:
#temporary pull of the file
emp_df.to_csv(r'C:\Users\m.bajaj\Python\Git_hub_repos\attrition-prediction\PII_safe_data\emp_df_merged.csv', index = False)